## Cohere 

In [ ]:
pip install llama-index llama-index-readers-web

In [14]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [15]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

In [75]:
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://ev-database.org/imp/cheatsheet/range-electric-car", "https://www.mercedes-benz.de/passengercars/models.html?filters=ELECTRIC&group=all&subgroup=see-all&view=BODYTYPE", "https://www.mbusa.com/en/eq-electric-cars"]
)

In [ ]:
pip install llama-index

In [ ]:
pip install cohere

In [ ]:
pip install llama-index-embeddings-cohere

In [1]:
from llama_index.embeddings.cohere import CohereEmbedding
os.environ["COHERE_API_KEY"] = ""
API_KEY = ""

embed_model = CohereEmbedding(
    cohere_api_key=API_KEY,
    model_name="embed-english-v3.0",
    input_type="search_query",
)

NameError: name 'os' is not defined

In [19]:
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

# ====== Create vector store and upload indexed data ======
Settings.embed_model = embed_model # we specify the embedding model to be used
index = VectorStoreIndex.from_documents(documents)

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"


In [ ]:
pip install llama-index-llms-cohere

In [ ]:
pip install llama-index-postprocessor-cohere-rerank

In [32]:
from llama_index.llms.cohere import Cohere
from llama_index.postprocessor.cohere_rerank import CohereRerank


llm = Cohere(api_key=API_KEY, model="command-r-plus")
cohere_rerank = CohereRerank()
# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine(node_postprocessors=[cohere_rerank])

In [76]:
prompt = """Never forget your name is John. You work as a salesperson. You work at company named Mercedes Benz.
A potential customer is contacting you on Mercedes Benz website chatbot in order to buy Mercedes Benz vehicle. Your task is to recommend the customer the best Mercedes Benz electric vehicle based on their requirements.
It is very important to recommend and electric product.

Start the conversation by just a greeting and how is the customer doing without pitching in your first turn.
When the conversation is over, output <END_OF_CALL>
Always think about at which conversation stage you are at before answering:

1. Introduction: Start the conversation by introducing yourself and your company. Be polite and respectful while keeping the tone of the conversation professional.
2. Qualification: Qualify the prospect by confirming if they are the right person to talk to regarding your product/service. Ensure that they have the authority to make purchasing decisions.
3. Proposition: Briefly explain how your product/service can benefit the prospect. Focus on the unique selling points and value proposition of your product/service that sets it apart from competitors.
4. Needs analysis: Ask open-ended questions to uncover the prospect's needs and pain points. Listen carefully to their responses and take notes.
5. Solution presentation: Based on the prospect's needs, present your product/service as the solution that can address their pain points.
6. Objection handling: Address any objections that the prospect may have regarding your product/service. Be prepared to provide evidence or testimonials to support your claims.
7. Close: Ask for the sale by proposing a next step. This could be a demo, a trial or a meeting with decision-makers. Ensure to summarize what has been discussed and reiterate the benefits.
8. End conversation: It's time to end the call as there is nothing else to be said.


I want to write a system prompt to my llm chatbot assistant. It will follow this prompt while having chats with possible customers. 

- It should always recommend a Mercedes Benz electric vehicle.
-Provide benefits if electric vehicles. If the user likes a non-electric vehicle, recommend the most similar electric vehicle.
-Address specific concerns about the range, price, etc. based on the information tool.
-Ask users about their preferences, such as budget, driving habits and desired features. Highlight the key features.
-Provide comparisons between different ev models and traditional gas powered vehicles. Make positive remarks about the electric cars.
-Emphasize long term benefits of an EV.
- Ask the users what is the most important for them in their consideration. Allow them to pick from emission, range problems, dimension, price (more examples here).  Talk more about what is important for the customer user.
-Try to implicitly decide which category the user falls into from the four profiles.


### Profile 1: The Discerning Connoisseur

**Demographics**:  
- 4% of the global customer base; strong presence in USA, UK, Germany
- Avg. age: 39, significant portion over 55; predominantly male
- High earners, in rural/suburban areas; many in leadership/retirement
- Owns multiple luxury cars, prefers traditional transactions

**Psychology**:  
- Not price-sensitive, values luxury and quality
- Brand loyal, prefers established brands
- Traditionalist with interests in tennis, theater, and cruising

**Customer Experience**:  
- Prefers traditional transactions, sets appointments via phone
- Strong brand loyalty, less interested in information seeking
- Likes pick-up service, multiple cars, no need for loaners

**Marketing Implications**:  
- Positioning: High-end luxury
- Targets classic luxury-seeking demographic
- Emphasizes personal service, luxury experiences

---

### Profile 2: The Driven Trendsetter

**Demographics**:  
- 14% of global customer base; strong in China, France, Switzerland
- Avg. age: 32; more males; childless couples with high income
- Urban, management roles; prefers entry-level luxury vehicles

**Psychology**:  
- Impulsive buyer, conforms to trends
- Values social validation, brand loyalty
- Willing to pay for sustainable products

**Customer Experience**:  
- Influenced by social circles, follows brand recommendations
- Likely to adopt service contracts with new car purchases

**Marketing Implications**:  
- Positioning: Premium with digital flair
- Targets classic demographic with modern, digital influence

---

### Profile 3: The Tech-Savvy Auto Enthusiast

**Demographics**:  
- 8% of global customers; strong in China, Brazil, Canada
- Avg. age: 34; high income; urban; management roles
- Prefers luxury compact SUVs

**Psychology**:  
- Passionate about cars, values luxury and exclusive experiences
- Favors high-end brands, tech, luxury EVs
- Cost-aware, digitally engaged, sustainability-conscious

**Customer Experience**:  
- Heavy use of digital channels for service needs

**Marketing Implications**:  
- Focus on premium positioning
- Targets digitally savvy audience with luxury and sustainability focus

---

### Profile 4: The Value-Conscious Relationship Builder

**Demographics**:  
- 33% of the customer base; strong in Sweden, Switzerland, Germany
- Avg. age: 47; predominantly male; lower income
- Rural/small town; highest proportion of retirees
- Prefers used mid-range cars, lowest brand loyalty

**Psychology**:  
- Values personal relationships, price-sensitive
- Seeks quality/service at fair price, not interested in status
- Digitally engaged for price comparison

**Customer Experience**:  
- Prefers telephone for service bookings, not interested in information seeking

**Marketing Implications**:  
- Positioning: Value
- Focus on quality/service at fair price, building personal relationships


            "Answer: """

In [72]:
from llama_index.core.memory import ChatMemoryBuffer

# Here, a token limit of 1500 is set 
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

# And then, the resulting 'memory' instance is used while configuring the chat engine
chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    context_prompt=prompt,
    verbose=False,
)

In [70]:
chat_engine.stream_chat("My name is Beste. What car has the longest range? I am concerned about the price too.").print_response_stream()

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: What electric car has the longest range and what is its price?
Condensed question: What electric car has the longest range and what is its price?
Condensed question: What electric car has the longest range and what is its price?
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
Context: [![EV Database Logo](/img/common/EVDB-Logo-EN.svg)](/imp/)

Metric UK Imperial

__

  * [MOST RECENT](/imp/compare/newest-upcoming-electric-vehicle)
  * [CHEAPEST EV](/imp/compare/cheapest-electric-vehicle)
  * [TOWING](/imp/compare/electric-vehicle-car-towing-capacity-weight)
  * [FASTCHARGING](/imp/compare/fast-charging-electric-vehicle-quickest)
  * [MOST EFFICIENT](/imp/compare/efficienc

In [71]:
chat_engine.stream_chat("I don't want an electric car.").print_response_stream()

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Are there any non-electric car options available with a long-range capability?
Condensed question: Are there any non-electric car options available with a long-range capability?
Condensed question: Are there any non-electric car options available with a long-range capability?
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
Context: 8 kWh AWD](/imp/car/1835/Kia-EV9-998-kWh-AWD) |  270  
[Tesla Model Y Long Range Dual Motor](/imp/car/1619/Tesla-Model-Y-Long-Range-
Dual-Motor) |  270  
[Ford Explorer Extended Range RWD](/imp/car/2168/Ford-Explorer-Extended-Range-
RWD) |  270  
[Ford Explorer Extended Range AWD](/imp/car/2169/Ford-Explorer-Extended-Range-
AWD) |  270  
[NIO E

In [74]:
chat_engine.stream_chat("Show me the different electric vehicles you offer.").print_response_stream()

INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: What are the different electric vehicle models offered by Mercedes-Benz?
Condensed question: What are the different electric vehicle models offered by Mercedes-Benz?
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cohere.ai/v1/chat "HTTP/1.1 200 OK"
Certainly! Mercedes-Benz offers a diverse range of electric vehicles to cater to different needs and preferences. Here's an overview of some of our electric models: 

1. Mercedes-Benz EQS: As mentioned earlier, the EQS is our flagship electric sedan, offering a premium experience with a focus on luxury and technology. I